In [2]:
import skyfield
from skyfield.api import load

planets = load('de421.bsp')
earth, mars = planets['earth'], planets['mars']

ts = load.timescale()
t = ts.now()
astrometric = earth.at(t).observe(mars)
ra, dec, distance = astrometric.radec()

print(ra)
print(dec)
print(distance)

[#################################] 100% de421.bsp


03h 58m 33.48s
+21deg 59' 51.8"
1.53581 au


In [3]:
from skyfield.api import N, W, wgs84

boston = earth + wgs84.latlon(42.3583 * N, 71.0636 * W)
astrometric = boston.at(t).observe(mars)
alt, az, d = astrometric.apparent().altaz()

print(alt)
print(az)

13deg 05' 00.6"
288deg 02' 51.2"


In [4]:
from astropy import units as u
xyz = astrometric.position.to(u.au)
altitude = alt.to(u.deg)

print(xyz)
print('{0:0.03f}'.format(altitude))

ModuleNotFoundError: No module named 'astropy'

In [5]:
from astropy import units as u
xyz = astrometric.position.to(u.au)
altitude = alt.to(u.deg)

print(xyz)
print('{0:0.03f}'.format(altitude))

[0.71977258 1.22870047 0.57523823] AU
13.084 deg


In [6]:
import datetime as dt
from pytz import timezone
from skyfield import almanac
from skyfield.api import N, W, wgs84, load

zone = timezone('US/Eastern')
now = zone.localize(dt.datetime.now())
midnight = now.replace(hour=0, minute=0, second=0, microsecond=0)
next_midnight = midnight + dt.timedelta(days=1)

ts = load.timescale()
t0 = ts.from_datetime(midnight)
t1 = ts.from_datetime(next_midnight)
eph = load('de421.bsp')
bluffton = wgs84.latlon(40.8939 * N, 83.8917 * W)

f = almanac.meridian_transits(eph, eph['Sun'], bluffton)
times, events = almanac.find_discrete(t0, t1, f)

# Select transits instead of antitransits.
times = times[events == 1]

t = times[0]
tstr = str(t.astimezone(zone))[:19]
print('Solar noon:', tstr)

Solar noon: 2021-03-07 12:46:27


In [7]:
import datetime as dt
from pytz import timezone
from skyfield import almanac
from skyfield.api import N, W, wgs84, load

# Figure out local midnight.
zone = timezone('US/Eastern')
now = zone.localize(dt.datetime.now())
midnight = now.replace(hour=0, minute=0, second=0, microsecond=0)
next_midnight = midnight + dt.timedelta(days=1)

ts = load.timescale()
t0 = ts.from_datetime(midnight)
t1 = ts.from_datetime(next_midnight)
eph = load('de421.bsp')
bluffton = wgs84.latlon(40.8939 * N, 83.8917 * W)
f = almanac.dark_twilight_day(eph, bluffton)
times, events = almanac.find_discrete(t0, t1, f)

for t, e in zip(times, events):
    tstr = str(t.astimezone(zone))[:16]
    print(tstr, ' ', almanac.TWILIGHTS[e], 'starts')

2021-03-07 05:28   Astronomical twilight starts
2021-03-07 06:00   Nautical twilight starts
2021-03-07 06:32   Civil twilight starts
2021-03-07 06:59   Day starts
2021-03-07 18:33   Civil twilight starts
2021-03-07 19:01   Nautical twilight starts
2021-03-07 19:33   Astronomical twilight starts
2021-03-07 20:05   Night starts


In [8]:
from skyfield.api import load
from skyfield.framelib import ecliptic_frame

ts = load.timescale()
t = ts.utc(2019, 12, 9, 15, 36)

eph = load('de421.bsp')
sun, moon, earth = eph['sun'], eph['moon'], eph['earth']

e = earth.at(t)
_, slon, _ = e.observe(sun).apparent().frame_latlon(ecliptic_frame)
_, mlon, _ = e.observe(moon).apparent().frame_latlon(ecliptic_frame)
phase = (mlon.degrees - slon.degrees) % 360.0

print('{0:.1f}'.format(phase))

149.4


In [9]:
import numpy as np
from skyfield.api import Angle, load

ts = load.timescale()
time = ts.utc(2020, 12, 30)

eph = load('de421.bsp')
earth, neptune = eph['earth'], eph['neptune barycenter']
radius_km = 24764.0

astrometric = earth.at(time).observe(neptune)
ra, dec, distance = astrometric.apparent().radec()
apparent_diameter = Angle(radians=np.arcsin(radius_km / distance.km) * 2.0)
print('{:.6f} arcseconds'.format(apparent_diameter.arcseconds()))

2.257190 arcseconds


In [10]:
from skyfield.api import load
from skyfield.framelib import ecliptic_frame
from skyfield.searchlib import find_maxima

ts = load.timescale()
t0 = ts.utc(2019)
t1 = ts.utc(2022)

eph = load('de421.bsp')
sun, earth, venus = eph['sun'], eph['earth'], eph['venus']

def elongation_at(t):
    e = earth.at(t)
    s = e.observe(sun).apparent()
    v = e.observe(venus).apparent()
    return s.separation_from(v).degrees

elongation_at.step_days = 15.0

times, elongations = find_maxima(t0, t1, elongation_at)

for t, elongation_degrees in zip(times, elongations):
    e = earth.at(t)
    _, slon, _ = e.observe(sun).apparent().frame_latlon(ecliptic_frame)
    _, vlon, _ = e.observe(venus).apparent().frame_latlon(ecliptic_frame)
    is_east = (vlon.degrees - slon.degrees) % 360.0 < 180.0
    direction = 'east' if is_east else 'west'
    print('{}  {:4.1f}° {} elongation'.format(
        t.utc_strftime(), elongation_degrees, direction))

2019-01-06 04:53:35 UTC  47.0° west elongation
2020-03-24 22:13:32 UTC  46.1° east elongation
2020-08-13 00:14:12 UTC  45.8° west elongation
2021-10-29 20:51:56 UTC  47.0° east elongation


In [11]:
from skyfield.api import N, W, load, wgs84
from skyfield.trigonometry import position_angle_of

ts = load.timescale()
t = ts.utc(2019, 9, 30, 23)

eph = load('de421.bsp')
sun, moon, earth = eph['sun'], eph['moon'], eph['earth']
boston = earth + wgs84.latlon(42.3583 * N, 71.0636 * W)

b = boston.at(t)
m = b.observe(moon).apparent()
s = b.observe(sun).apparent()
print(position_angle_of(m.altaz(), s.altaz()))

238deg 55' 55.3"
